In [608]:
import pickle


resumes = pickle.load( open( "all_resumes.p", "rb" ) )
JD = pickle.load( open( "JD.p", "rb" ) )
pickle_files = !ls -d '/home/mandub/Desktop/Thakaa/Resumes-Filtering/JD_resume_distance/'*.p
JD_resume_distance_dict={}
for pickle_file in pickle_files:
    PFkey = int(pickle_file.split('/')[-1].replace(".p",''))
    JD_resume_distance_dict[PFkey]=pickle.load(
        open( "JD_resume_distance/" +pickle_file.split('/')[-1], "rb" ))
print (len(JD_resume_distance_dict))

753


In [609]:
def Sort_Distance(dict_distance):
    sorted_distance = {} 
    for key in dict_distance:
        alist = dict_distance[key]
        alist.sort(key = lambda x: x[1])
        sorted_distance[key] = alist[:100]        
    return sorted_distance

sorted_distance = Sort_Distance(JD_resume_distance_dict)


In [610]:
#print (sorted_distance[4])

In [611]:
import json
with open('./assets/sorted_distance.txt', 'w') as outfile:
    json.dump(sorted_distance, outfile)

In [1038]:
from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input
import plotly.graph_objs as go
from flask import Flask
import flask
import webbrowser
import os
import pandas as pd
import dash_bootstrap_components as dbc
from dash.dependencies import State
import dash_table
#####################
import nltk
import numpy
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
stop_words = set (stopwords.words("english"))

In [1039]:
#STATIC_PATH = os.path.join(os.path.dirname(os.path.abspath("static")), 'static')
#server = Flask(__name__)
#app = dash.Dash(name = __name__, server = server)


In [1040]:
css_sheet = "./assets/style.css"

external_stylesheets=[dbc.themes.BOOTSTRAP]
app = JupyterDash(__name__,external_stylesheets=external_stylesheets)



alist = [1,2,3]
header_bar = html.Div([html.H1("Human Resource"),
                       html.Img(src="./assets/thakaa_logo.png", width=200)],
                      className= "header-style")

jd_filter_div = html.Div([html.Label("Select Job Describtion", className= "input-style"),
                                dcc.Input(id="jd-input",value="", placeholder="Search....",
                                         className= "input-style"), html.Div(id="jd-output")], className="block-style")
'''
Category_div = html.Div([    html.Button('Job Category', id='btn1', n_clicks=0 ),
    html.Button('Button 2', id='btn2', n_clicks=0),
    html.Button('Button 3', id='btn3', n_clicks=0),
    html.Div(id='Category_div1')],className="header-style")
''' 

'\nCategory_div = html.Div([    html.Button(\'Job Category\', id=\'btn1\', n_clicks=0 ),\n    html.Button(\'Button 2\', id=\'btn2\', n_clicks=0),\n    html.Button(\'Button 3\', id=\'btn3\', n_clicks=0),\n    html.Div(id=\'Category_div1\')],className="header-style")\n'

In [1041]:
dominant_topics_df = pd.DataFrame(sorted_distance) 

In [1042]:
n_documents = dbc.Col([
   html.Div([
        html.H4(len(alist)),
        html.H5("n documents")
   ], className = "m-2 p-3 border border-light rounded bg-secondary text-white")
], md=5)

value_counts = dominant_topics_df[0].value_counts()
df_topics_count = pd.DataFrame({"Highest_Score_Topic": value_counts.index, "count" : value_counts.values})
#df_topics_count.head(10)


In [1043]:
# JD
print(JD[0][0],JD[0][1],JD[0][2],JD[0][3],JD[4][0])
Category = {}
for index, item in enumerate(JD):
    if item[1] not in Category:
        Category[item[1]]= [[index,item[2]]]
    else:
        Category[item[1]].append([index,item[2]])
    
    
#Category_pd = pd.DataFrame(Category)

﻿No Category Job What is 4


In [1044]:
import dash_core_components as dcc
dropdownCategory = dcc.Dropdown(
                id='dropdownCategory',
                options=[ {"label":topic, "value": topic} for topic in Category],
                #label="Category",
                value='Category' )
dropdownjob = dcc.Dropdown(
                id='dropdownjob',
                options=[ {"label":"a job", "value": "a job"} ],
                value='Select_Job' )


In [1045]:

controls = dbc.Card(
    [
        html.H2("Job Category"),
        html.Hr(),
        dbc.Label("Select a Category"),dropdownCategory ,
        html.H2("Select a Job"),
        html.Hr(),
        dbc.Label("Select a Job"),#dropdownjob ,
        #dcc.Upload(children= [dbc.Button('Upload File', color="primary", className="mr-1")],
        #id='upload-data',
        #Allow multiple files to be uploaded
        #multiple=False
        #),
        html.Div(id='output1', children="Output:-", className="my-2"),
        html.Hr(),
        dbc.Button('Search for a Candidate  Resume', color="primary", className="mr-1"),
        html.Div(id='output2', children="Choose a Job", className="my-2"),
        dbc.Button('Search for a Candidate  Resume', color="primary", className="mr-1"),
        html.Hr(),
        html.Div(id='output3', children="output333", className="my-2"),
        #dash_table.DataTable( id='table1' )
        
        
    ],       
    body=True,
        
)

row = dbc.Row(
    [dbc.Col(controls, md=4)],
    className="my-3")#'''dbc.Col(statisics_widget, md=8),'''

In [1046]:
row = dbc.Row([
    dbc.Col(controls, md=8)],
    className="my-3")


In [1047]:
def Resumes_words_sents_tokenize_filter(resume):
    words = word_tokenize(resume)
    filter_resume = [w for w in words if not w in stop_words]
    Astring = ''
    for i in filter_resume:
        Astring= Astring+' '+ str(i)
    return Astring 

In [1048]:
#app.layout = html.Div(children=[header_bar,jd_filter_div, Category_div])#header_bar,jd_filter_div,
#fluid=true ensures the containers take 100% screen size
app.layout = dbc.Container([header_bar,html.Hr(),row], fluid=True) #

In [1049]:
@app.callback( Output(component_id ="output1" , component_property = "children"),
              [Input(component_id = "dropdownCategory", component_property = "value")])

def update_jobs(Select_Category):
    jobs = Category[Select_Category]
    options=[{'label':job[1], 'value':job[0]} for job in jobs]
    dropdownjob = dcc.Dropdown(
                id='dropdownjob',
                options= options,
                value='Select_Job' )
    return dropdownjob #"Output changed: {}".format(Select_Job)
    

In [1050]:
@app.callback(Output(component_id = 'output2', component_property ='children'),
    [Input('dropdownjob', component_property ='value')])
def show_job(job_id):
    job_info = [html.H4("Job Descriptions: - {}".format(JD[job_id][3])),html.Hr(),
            html.H4("What Does: - {}".format(JD[job_id][4])),html.Hr(),
            html.H4("Job Brief: - {}".format(JD[job_id][6])),html.Hr()            
           ]
    #job_dsc = 'Job Descriptions:' + JD[job_id][3]+' '+'What Does'+JD[job_id][4] +'\n'+'Job brief'+JD[job_id][6]
    return job_info


In [1051]:
# resumes[[ID][Source][Path][file_name][Extension][Resume_String]{Resume_dict}]
@app.callback(Output(component_id = 'output3', component_property ='children'),
    [Input('dropdownjob', component_property ='value')])
def show_Candidate_Resumes(job_id):
    alist= sorted_distance[job_id]
    Candidate_Resume= alist[:10]
    C_R_L =[]
    for i in Candidate_Resume:
        resumeID= resumes[i[0]][0]
        resumePATH= resumes[i[0]][2] +resumes[i[0]][3]
        resumestr=Resumes_words_sents_tokenize_filter(resumes[i[0]][5]) 
        C_R_L.append("resumeID: "+str(resumeID)+" resumePATH "+ resumePATH +"-------"+ resumestr)

    canadidate_info= [
        html.H4("A Resume 1: - {}".format(C_R_L[0])),html.Hr(),
        html.H4("A Resume 2: - {}".format(C_R_L[1])),html.Hr(),
        html.H4("A Resume 3: - {}".format(C_R_L[2])),html.Hr(),
        html.H4("A Resume 4: - {}".format(C_R_L[3])),html.Hr(),
        html.H4("A Resume 5: - {}".format(C_R_L[4])),html.Hr(),
        html.H4("A Resume 6: - {}".format(C_R_L[5])),html.Hr(),
        html.H4("A Resume 7: - {}".format(C_R_L[6])),html.Hr(),
        html.H4("A Resume 8: - {}".format(C_R_L[7])),html.Hr(),
        html.H4("A Resume 9: - {}".format(C_R_L[8])),html.Hr(),
        html.H4("A Resume10: - {}".format(C_R_L[9])),html.Hr()
    ]
    return canadidate_info
    

In [1052]:

if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
